In [ ]:
from google.colab import drive

PATH_DRIVE = '/content/drive'
drive.mount(PATH_DRIVE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd

from datetime import datetime
from pathlib import Path

PATH_BASE = Path(PATH_DRIVE)/'MyDrive'
print(os.listdir(PATH_BASE))

['문서', '사진', '설문', 'drawio', 'Colab Notebooks', 'data', 'models', 'events.out.tfevents.1675508059.c6da5a757682.406.0', '프로젝트 서론의 사본.gdoc']


In [ ]:
! pip3 install -U datasets transformers evaluate sentencepiece

# import torch
import datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from evaluate import load

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
now = datetime.now()
timestamp = datetime.timestamp(now)

NAME_MODEL = 'cross-encoder/nli-deberta-v3-base'
NAME_DIR = f"mednli_{NAME_MODEL.split('/')[-1]}_{int(timestamp)}"

PATH_MODELS = PATH_BASE/'models'/NAME_DIR

dataset = datasets.load_dataset(
    'bigbio/mednli', 
    'mednli_source',
    data_dir=PATH_BASE/'data'
)
model = AutoModelForSequenceClassification.from_pretrained(NAME_MODEL)
tokenizer = AutoTokenizer.from_pretrained(NAME_MODEL)

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def label_encode(data):
    labels = {'contradiction': 0, 'entailment': 1, 'neutral': 2}
    key = data['gold_label']
    data = {'label': labels[key]}

    return data


def tokenize(data):
    data = tokenizer(
        data['sentence1'],
        data['sentence2'],
        max_length=model.config.max_position_embeddings,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    return data


def compute_metrics(y):
    logits, labels = y
    preds = logits.argmax(axis=1)
    metrics = 'precision', 'recall', 'f1'
    result = {}

    result.update(
        load('accuracy').compute(
            predictions=preds,
            references=labels
        )
    )

    for metric in metrics:
        result.update(
            load(metric).compute(
                predictions=preds,
                references=labels, 
                average='macro'
            )
        )

    return result


In [ ]:
cols = [
    'input_ids', 
    'token_type_ids',
    'attention_mask', 
    'label'
]
cols = [col for col in dataset.column_names['test'] if col not in cols]

dataset = dataset.map(label_encode)
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.remove_columns(cols)
dataset = dataset.shuffle(seed=23)

  0%|          | 0/11232 [00:00<?, ?ex/s]

  0%|          | 0/1422 [00:00<?, ?ex/s]

  0%|          | 0/1395 [00:00<?, ?ex/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
numEpoch = 10
numBatch = 8
numStep = dataset.num_rows['train'] * numEpoch / numBatch

args = TrainingArguments(
    run_name=NAME_DIR,
    num_train_epochs=numEpoch,
    per_device_train_batch_size=numBatch,
    per_device_eval_batch_size=numBatch*4,
    gradient_accumulation_steps=8,
    # eval_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=1e-2,
    warmup_ratio=1e-2,
    warmup_steps=int(numStep/100),
    save_total_limit=3,
    save_steps=200,
    logging_steps=50,
    save_strategy='steps',
    evaluation_strategy='steps',
    output_dir=PATH_MODELS/'weights',
    logging_dir=PATH_MODELS/'logs',
    seed=23,
    metric_for_best_model='f1',
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
if not os.path.exists(PATH_MODELS):
    os.mkdir(PATH_MODELS)

with open(PATH_MODELS/'args.json', 'w', encoding='utf-8') as f:
    text = args.to_json_string()
    f.write(text)

In [ ]:
result = trainer.evaluate(dataset['test'])
df = pd.DataFrame.from_dict([result])

df.to_csv(PATH_MODELS/'pre_result.csv', index=False, encoding='utf-8')
df

***** Running Evaluation *****
  Num examples = 1422
  Batch size = 32


,eval_loss,eval_accuracy,eval_precision,eval_recall,eval_f1,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,1.402204,0.720816,0.753227,0.720816,0.717678,69.9021,20.343,0.644


In [ ]:
trainer.train()
trainer.save_model(PATH_MODELS/'weights')

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11232
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 1750
  Number of trainable parameters = 184424451


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.013500,0.691494,0.792115,0.792441,0.792115,0.792273
100,0.605700,0.463934,0.807885,0.809635,0.807885,0.808324
150,0.500500,0.433133,0.822939,0.827341,0.822939,0.822101
200,0.465700,0.422840,0.834409,0.836378,0.834409,0.834150
250,0.389000,0.395714,0.850896,0.851101,0.850896,0.850994
300,0.364000,0.417404,0.846595,0.846998,0.846595,0.846616
350,0.387500,0.381474,0.849462,0.850730,0.849462,0.849371
400,0.271200,0.407548,0.855197,0.857161,0.855197,0.855871
450,0.288300,0.400774,0.858781,0.861467,0.858781,0.859265
500,0.297200,0.395891,0.857348,0.857874,0.857348,0.857169


***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32


***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/models/mednli_nli-deberta-v3-base_1675645855/weights/checkpoint-200
Configuration saved in /content/drive/MyDrive/models/mednli_nli-deberta-v3-base_1675645855/weights/checkpoint-200/config.json
Model weights saved in /content/drive/MyDrive/models/mednli_nli-deberta-v3-base_1675645855/weights/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1395
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/models/mednli_nli-deberta-v3-base_1675645855/weights/checkpoint-400
Configura

In [ ]:
result = trainer.evaluate(dataset['test'])
df = pd.DataFrame.from_dict([result])

df.to_csv(PATH_MODELS/'result.csv', index=False, encoding='utf-8')
df

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/models/